In [71]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import json
import re


"https://www.petfinder.com"

In [74]:
def adoptable_dogs(URL, location):
    try:
        # Setting up webdriver
        driver = webdriver.Chrome()

        # Navigating to the website
        driver.get(URL)

        # Giving website some time to load (fixing time)
        time.sleep(2)

        # Having the driver wait animal type search bar is visible (searching for element by its id)
        form_type = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, 'simpleSearchAnimalType'))
        )
        # Clearing the search bar just in case there is previous text 
        form_type.clear()
        # Inputing "Dogs" into the search bar for animal type 
        form_type.send_keys("Dogs")

        
        # Searching for the location search bar element by its id
        form_location = driver.find_element(By.ID, 'simpleSearchLocation') 

        # form_location = WebDriverWait(driver, 10).until(
        #     EC.element_to_be_clickable((By.ID, 'simpleSearchLocation'))
        # )
        # Clearing the search bar just in case there is previous text 
        form_location.clear()
        # Inputing the location parameter into the search bar for animal type 
        form_location.send_keys(location)
        
        # Giving the location a second to settle into the search bar  
        time.sleep(2)


        search_button = driver.find_element(By.ID, 'petSearchBarSearchButton') 
        # Click the search button
        # search_button = WebDriverWait(driver, 10).until(
        #     EC.element_to_be_clickable((By.ID, 'petSearchBarSearchButton'))
        # )
        search_button.click()

    except Exception as e:
        # Custom exception message with the actual error that occurred
        print(f"An error occurred while trying to search for adoptable dogs: {str(e)}")


In [68]:
def get_info(Driver):
# Step 1: Locate the dog cards
    dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

    # Limit the number of dogs to process
    max_dogs = 10
    processed_dogs = 0  # Keep track of how many dogs have been processed

    # Step 2: Loop through each card, click on it, extract info, and go back (limit to 10)
    while processed_dogs < max_dogs:
        try:
            # Step 2a: Re-locate the dog cards on each loop iteration to avoid stale references
            dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

            # Click on the dog card based on the index
            card = dog_cards[processed_dogs]
            
            # Scroll into view and click on the card
            driver.execute_script("arguments[0].scrollIntoView();", card)
            card.click()

            # Step 3: Wait for the dog's detailed page to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//h1[@data-test="Pet_Detail_Block"]'))
            )

            # Step 4: Extract relevant information with try-except blocks to handle missing data
            try:
                pet_name = driver.find_element(By.XPATH, '//span[@data-test="Pet_Name"]').text
            except:
                pet_name = 'N/A'

            try:
                pet_breeds = driver.find_element(By.XPATH, '//span[@data-test="Pet_Breeds"]').text
            except:
                pet_breeds = 'N/A'

            try:
                pet_location = driver.find_element(By.XPATH, '//span[@data-test="Pet_Location"]').text
            except:
                pet_location = 'N/A'

            try:
                pet_age = driver.find_element(By.XPATH, '//span[@data-test="Pet_Age"]').text
            except:
                pet_age = 'N/A'

            try:
                pet_sex = driver.find_element(By.XPATH, '//span[@data-test="Pet_Sex"]').text
            except:
                pet_sex = 'N/A'

            try:
                pet_size = driver.find_element(By.XPATH, '//span[@data-test="Pet_Full_Grown_Size"]').text
            except:
                pet_size = 'N/A'

            try:
                pet_color = driver.find_element(By.XPATH, '//span[@data-test="Pet_Primary_Color"]').text
            except:
                pet_color = 'N/A'

            try:
                characteristics = driver.find_element(By.XPATH, '//dt[contains(text(), "Characteristics")]/following-sibling::dd').text
            except:
                characteristics = 'N/A'

            try:
                coat_length = driver.find_element(By.XPATH, '//dt[contains(text(), "Coat length")]/following-sibling::dd').text
            except:
                coat_length = 'N/A'

            try:
                house_trained = driver.find_element(By.XPATH, '//dt[contains(text(), "House-trained")]/following-sibling::dd').text
            except:
                house_trained = 'N/A'

            try:
                health = driver.find_element(By.XPATH, '//dt[contains(text(), "Health")]/following-sibling::dd').text
            except:
                health = 'N/A'

            try:
                prefers_without = driver.find_element(By.XPATH, '//dt[contains(text(), "Prefers a home without")]/following-sibling::dd').text
            except:
                prefers_without = 'N/A'

            try:
                prefers_with = driver.find_element(By.XPATH, '//dt[contains(text(), "Good in a home with")]/following-sibling::dd').text
            except:
                prefers_with = 'N/A'

            try:
                adoption_fee = driver.find_element(By.XPATH, '//dt[contains(text(), "Adoption fee")]/following-sibling::dd').text
            except:
                adoption_fee = 'N/A'

            # Print or store the extracted details
            print(f"Pet Name: {pet_name}")
            print(f"Breeds: {pet_breeds}")
            print(f"Location: {pet_location}")
            print(f"Age: {pet_age}")
            print(f"Sex: {pet_sex}")
            print(f"Size: {pet_size}")
            print(f"Color: {pet_color}")
            print(f"Characteristics: {characteristics}")
            print(f"Coat Length: {coat_length}")
            print(f"House-Trained: {house_trained}")
            print(f"Health: {health}")
            print(f"Prefers a Home Without: {prefers_without}")
            print("=" * 40)

            # Step 5: Navigate back to the search results page
            driver.back()

            # Wait for the results page to reload
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//a[@class="petCard-link"]'))
            )

            # Increment the processed dogs counter
            processed_dogs += 1

        except Exception as e:
            print(f"Error processing dog {processed_dogs}: {e}")
            break

# Close the driver once done
driver.quit()

Pet Name: Jackson
Breeds: Boxer & American Bulldog Mix
Location: Atlanta , GA
Age: Senior
Sex: Male
Size: Medium
Color: Brindle
Characteristics: Friendly, Affectionate, Smart, Athletic, Food Motivated, Dog Reactive When On Leash
Coat Length: Short
House-Trained: Yes
Health: Vaccinations up to date, spayed / neutered.
Prefers a Home Without: Other dogs, cats.
Pet Name: Marshmallow
Breeds: Great Pyrenees Mix
Location: Atlanta , GA
Age: Puppy
Sex: Female
Size: Large
Color: N/A
Characteristics: N/A
Coat Length: N/A
House-Trained: N/A
Health: Vaccinations up to date.
Prefers a Home Without: N/A
Pet Name: Candy
Breeds: Chiweenie Mix
Location: Atlanta , GA
Age: Puppy
Sex: Female
Size: Small
Color: N/A
Characteristics: N/A
Coat Length: N/A
House-Trained: N/A
Health: Vaccinations up to date.
Prefers a Home Without: N/A
Pet Name: Ezra - Pending
Breeds: Italian Greyhound & American Bulldog Mix
Location: Atlanta, GA
Age: Adult
Sex: Male
Size: Medium
Color: N/A
Characteristics: N/A
Coat Length: N/A

In [73]:
# Step 1: Locate the dog cards
dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

# Limit the number of dogs to process (adjust as needed)
max_dogs = 10
processed_dogs = 0  # Counter to track processed dogs

# Step 2: Loop through each card, click on it, extract info, and go back (limit to max_dogs)
while processed_dogs < max_dogs:
    try:
        # Step 2a: Re-locate the dog cards to avoid stale elements
        dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

        # Click on the dog card based on the index
        card = dog_cards[processed_dogs]

        # Scroll into view and click on the card
        driver.execute_script("arguments[0].scrollIntoView();", card)
        card.click()

        # Step 3: Wait for the dog's detailed page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//h1[@data-test="Pet_Detail_Block"]'))
        )

        # Step 4: Locate the <pf-ad> element containing dog details
        pf_ad = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//pf-ad[contains(@id, "PetDetail")]'))
        )

        # Step 5: Extract the 'targeting' attribute and parse it
        targeting_data = pf_ad.get_attribute("targeting")
        decoded_data = html.unescape(targeting_data)

        # Parse the decoded data into a dictionary
        dog_info = json.loads(decoded_data)

        # Step 6: Extract individual fields from the dictionary (using get() for missing data)
        pet_id = dog_info.get('Pet_ID', 'N/A')
        pet_name = dog_info.get('Pet_Name', 'N/A')
        pet_type = dog_info.get('Type', 'N/A')
        species = dog_info.get('Species', 'N/A')
        primary_breed = dog_info.get('Primary_Breed', 'N/A')
        secondary_breed = dog_info.get('Secondary_Breed', 'N/A')
        mixed_breed = dog_info.get('Mixed_Breed', 'N/A')
        age = dog_info.get('Age', 'N/A')
        gender = dog_info.get('Gender', 'N/A')
        characteristics = dog_info.get('Characteristics', 'N/A')
        size = dog_info.get('Size', 'N/A')
        primary_color = dog_info.get('Primary_color', 'N/A')
        coat_length = dog_info.get('Coat_length', 'N/A')
        shelter_name = dog_info.get('Shelter_Name', 'N/A')
        shelter_id = dog_info.get('Shelter_ID', 'N/A')
        zip_code = dog_info.get('Zip_Code', 'N/A')
        adoption_fee = dog_info.get('Adoption_fee', 'N/A')

        # Print the extracted information (you can save or store it as needed)
        print(f"Pet ID: {pet_id}")
        print(f"Pet Name: {pet_name}")
        print(f"Type: {pet_type}")
        print(f"Species: {species}")
        print(f"Primary Breed: {primary_breed}")
        print(f"Secondary Breed: {secondary_breed}")
        print(f"Mixed Breed: {mixed_breed}")
        print(f"Age: {age}")
        print(f"Gender: {gender}")
        print(f"Characteristics: {characteristics}")
        print(f"Size: {size}")
        print(f"Primary Color: {primary_color}")
        print(f"Coat Length: {coat_length}")
        print(f"Shelter Name: {shelter_name}")
        print(f"Shelter ID: {shelter_id}")
        print(f"Zip Code: {zip_code}")
        print(f"Adoption Fee: {adoption_fee}")
        print("=" * 40)

        # Step 7: Navigate back to the search results page
        driver.back()

        # Wait for the search results page to reload
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="petCard-link"]'))
        )

        # Increment the processed dogs counter
        processed_dogs += 1

    except Exception as e:
        print(f"Error processing dog {processed_dogs}: {e}")
        break

# Close the driver once done
driver.quit()

Error processing dog 0: 'str' object has no attribute 'unescape'


In [75]:

# Step 1: Locate the dog cards
dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

# Limit the number of dogs to process (adjust as needed)
max_dogs = 10
processed_dogs = 0  # Counter to track processed dogs

# Step 2: Loop through each card, click on it, extract info, and go back (limit to max_dogs)
while processed_dogs < max_dogs:
    try:
        # Step 2a: Re-locate the dog cards to avoid stale elements
        dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

        # Click on the dog card based on the index
        card = dog_cards[processed_dogs]

        # Scroll into view and click on the card
        driver.execute_script("arguments[0].scrollIntoView();", card)
        card.click()

        # Step 3: Wait for the dog's detailed page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//h1[@data-test="Pet_Detail_Block"]'))
        )

        # Step 4: Locate the <pf-ad> element containing dog details
        pf_ad = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//pf-ad[contains(@id, "PetDetail")]'))
        )

        # Step 5: Extract the 'targeting' attribute and parse it (no unescape needed)
        targeting_data = pf_ad.get_attribute("targeting")

        # Parse the targeting data directly into a dictionary
        dog_info = json.loads(targeting_data)

        # Step 6: Extract individual fields from the dictionary (using get() for missing data)
        pet_id = dog_info.get('Pet_ID', 'N/A')
        pet_name = dog_info.get('Pet_Name', 'N/A')
        pet_type = dog_info.get('Type', 'N/A')
        species = dog_info.get('Species', 'N/A')
        primary_breed = dog_info.get('Primary_Breed', 'N/A')
        secondary_breed = dog_info.get('Secondary_Breed', 'N/A')
        mixed_breed = dog_info.get('Mixed_Breed', 'N/A')
        age = dog_info.get('Age', 'N/A')
        gender = dog_info.get('Gender', 'N/A')
        characteristics = dog_info.get('Characteristics', 'N/A')
        size = dog_info.get('Size', 'N/A')
        primary_color = dog_info.get('Primary_color', 'N/A')
        coat_length = dog_info.get('Coat_length', 'N/A')
        shelter_name = dog_info.get('Shelter_Name', 'N/A')
        shelter_id = dog_info.get('Shelter_ID', 'N/A')
        zip_code = dog_info.get('Zip_Code', 'N/A')


            try:
                pet_story_section = driver.find_element(By.XPATH, '//div[@data-test="Pet_Story_Section"]').text
                # Use regular expression to find "Adoption fee" followed by a number
                adoption_fee_search = re.search(r'Adoption fee (\d+)', pet_story_section, re.IGNORECASE)
                if adoption_fee_search:
                    adoption_fee = adoption_fee_search.group(1)
                else:
                    adoption_fee = "Not Available"
            except:
                adoption_fee = "Not Available"

        
        try:
            adoption_fee = driver.find_element(By.XPATH, '//dt[contains(text(), "Adoption fee")]/following-sibling::dd').text
        except:
            adoption_fee = 'N/A'


        # Print the extracted information (you can save or store it as needed)
        print(f"Pet ID: {pet_id}")
        print(f"Pet Name: {pet_name}")
        print(f"Type: {pet_type}")
        print(f"Species: {species}")
        print(f"Primary Breed: {primary_breed}")
        print(f"Secondary Breed: {secondary_breed}")
        print(f"Mixed Breed: {mixed_breed}")
        print(f"Age: {age}")
        print(f"Gender: {gender}")
        print(f"Characteristics: {characteristics}")
        print(f"Size: {size}")
        print(f"Primary Color: {primary_color}")
        print(f"Coat Length: {coat_length}")
        print(f"Shelter Name: {shelter_name}")
        print(f"Shelter ID: {shelter_id}")
        print(f"Zip Code: {zip_code}")
        print("=" * 40)

        # Step 7: Navigate back to the search results page
        driver.back()

        # Wait for the search results page to reload
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[@class="petCard-link"]'))
        )

        # Increment the processed dogs counter
        processed_dogs += 1

    except Exception as e:
        print(f"Error processing dog {processed_dogs}: {e}")
        break

# Close the driver once done
driver.quit()

Pet ID: 72096072
Pet Name: Jackson
Type: Dog
Species: Dog
Primary Breed: Boxer
Secondary Breed: American Bulldog
Mixed Breed: True
Age: Senior
Gender: Male
Characteristics: House trained,Spay/Neuter,Shots Current
Size: Medium
Primary Color: Brindle
Coat Length: Short
Shelter Name: Freedom Bridge Animal Rescue
Shelter ID: GA988
Zip Code: 30303
Adoption Fee: consumer.animal_detail.adoption_fee.label.fee
Pet ID: 73529097
Pet Name: Marshmallow
Type: Dog
Species: Dog
Primary Breed: Great Pyrenees
Secondary Breed: 
Mixed Breed: True
Age: Baby
Gender: Female
Characteristics: Shots Current
Size: Large
Primary Color: 
Coat Length: 
Shelter Name: Carolina German Shepherd Rescue
Shelter ID: SC425
Zip Code: 30303
Adoption Fee: consumer.animal_detail.adoption_fee.label.fee
Pet ID: 73491201
Pet Name: Bowie
Type: Dog
Species: Dog
Primary Breed: Labrador Retriever
Secondary Breed: Mixed Breed
Mixed Breed: True
Age: Baby
Gender: Male
Characteristics: House trained,Shots Current
Size: Large
Primary Colo

In [ ]:
def get_info(Driver):
# Step 1: Locate the dog cards
    dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

    # Limit the number of dogs to process
    max_dogs = 10
    processed_dogs = 0  # Keep track of how many dogs have been processed

    # Step 2: Loop through each card, click on it, extract info, and go back (limit to 10)
    while processed_dogs < max_dogs:
        try:
            # Step 2a: Re-locate the dog cards on each loop iteration to avoid stale references
            dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

            # Click on the dog card based on the index
            card = dog_cards[processed_dogs]
            
            # Scroll into view and click on the card
            driver.execute_script("arguments[0].scrollIntoView();", card)
            card.click()

            # Step 3: Wait for the dog's detailed page to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//h1[@data-test="Pet_Detail_Block"]'))
            )

            # Step 4: Extract relevant information with try-except blocks to handle missing data
            try:
                pet_name = driver.find_element(By.XPATH, '//span[@data-test="Pet_Name"]').text
            except:
                pet_name = 'N/A'

            try:
                pet_breeds = driver.find_element(By.XPATH, '//span[@data-test="Pet_Breeds"]').text
            except:
                pet_breeds = 'N/A'

            try:
                pet_location = driver.find_element(By.XPATH, '//span[@data-test="Pet_Location"]').text
            except:
                pet_location = 'N/A'

            try:
                pet_age = driver.find_element(By.XPATH, '//span[@data-test="Pet_Age"]').text
            except:
                pet_age = 'N/A'

            try:
                pet_sex = driver.find_element(By.XPATH, '//span[@data-test="Pet_Sex"]').text
            except:
                pet_sex = 'N/A'

            try:
                pet_size = driver.find_element(By.XPATH, '//span[@data-test="Pet_Full_Grown_Size"]').text
            except:
                pet_size = 'N/A'

            try:
                pet_color = driver.find_element(By.XPATH, '//span[@data-test="Pet_Primary_Color"]').text
            except:
                pet_color = 'N/A'

            try:
                characteristics = driver.find_element(By.XPATH, '//dt[contains(text(), "Characteristics")]/following-sibling::dd').text
            except:
                characteristics = 'N/A'

            try:
                coat_length = driver.find_element(By.XPATH, '//dt[contains(text(), "Coat length")]/following-sibling::dd').text
            except:
                coat_length = 'N/A'

            try:
                house_trained = driver.find_element(By.XPATH, '//dt[contains(text(), "House-trained")]/following-sibling::dd').text
            except:
                house_trained = 'N/A'

            try:
                health = driver.find_element(By.XPATH, '//dt[contains(text(), "Health")]/following-sibling::dd').text
            except:
                health = 'N/A'

            try:
                prefers_without = driver.find_element(By.XPATH, '//dt[contains(text(), "Prefers a home without")]/following-sibling::dd').text
            except:
                prefers_without = 'N/A'

            try:
                prefers_with = driver.find_element(By.XPATH, '//dt[contains(text(), "Good in a home with")]/following-sibling::dd').text
            except:
                prefers_with = 'N/A'

            try:
                adoption_fee = driver.find_element(By.XPATH, '//dt[contains(text(), "Adoption fee")]/following-sibling::dd').text
            except:
                adoption_fee = 'N/A'

            # Print or store the extracted details
            print(f"Pet Name: {pet_name}")
            print(f"Breeds: {pet_breeds}")
            print(f"Location: {pet_location}")
            print(f"Age: {pet_age}")
            print(f"Sex: {pet_sex}")
            print(f"Size: {pet_size}")
            print(f"Color: {pet_color}")
            print(f"Characteristics: {characteristics}")
            print(f"Coat Length: {coat_length}")
            print(f"House-Trained: {house_trained}")
            print(f"Health: {health}")
            print(f"Prefers a Home Without: {prefers_without}")
            print("=" * 40)

            # Step 5: Navigate back to the search results page
            driver.back()

            # Wait for the results page to reload
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//a[@class="petCard-link"]'))
            )

            # Increment the processed dogs counter
            processed_dogs += 1

        except Exception as e:
            print(f"Error processing dog {processed_dogs}: {e}")
            break

# Close the driver once done
driver.quit()

In [ ]:
def get_info(driver):
    try:

        # Ensuring the dog cards have time to load and be clicked upon 
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@class="petCard-link"]'))
        )
        dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')

        print("CHECK 1")

        # Limit the number of dogs to process
        max_dogs = 10
        processed_dogs = 0

        dog_data_list = []

        # Looping through each dog card to extract information 
        while processed_dogs < max_dogs:
            try:
                # Re-finding the dog cards to avoid stale references
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//a[@class="petCard-link"]'))
                )
                dog_cards = driver.find_elements(By.XPATH, '//a[@class="petCard-link"]')
                print("CHECK 2")
                # Click on the current dog card
                card = dog_cards[processed_dogs]
                print("CHECK 3")
                driver.execute_script("arguments[0].scrollIntoView();", card)
                card.click()

                # Wait for the dog's detailed page to load
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//h1[@data-test="Pet_Detail_Block"]'))
                )

                # Finding the <pf-ad> element that contains the dogs details 
                pf_ad = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//pf-ad[contains(@id, "PetDetail")]'))
                )

                # Extracting 'targeting' attribute and load to a json file 
                targeting_data = pf_ad.get_attribute("targeting")
                dog_info = json.loads(targeting_data)

                # Extracted desired field names from dog_info 
                pet_id = dog_info.get('Pet_ID', 'N/A')
                pet_name = dog_info.get('Pet_Name', 'N/A')
                primary_breed = dog_info.get('Primary_Breed', 'N/A')
                secondary_breed = dog_info.get('Secondary_Breed', 'N/A')
                mixed_breed = dog_info.get('Mixed_Breed', 'N/A')
                age = dog_info.get('Age', 'N/A')
                gender = dog_info.get('Gender', 'N/A')
                size = dog_info.get('Size', 'N/A')
                primary_colour = dog_info.get('Primary_color', 'N/A')
                secondary_colour = dog_info.get('Secondary_color', 'N/A')
                coat_length = dog_info.get('Coat_length', 'N/A')
                shelter_name = dog_info.get('Shelter_Name', 'N/A')
                shelter_id = dog_info.get('Shelter_ID', 'N/A')
                zip_code = dog_info.get('Zip_Code', 'N/A')
                num_photos = dog_info.get('Number_of_photos_in_profile', 'N/A')
                children = dog_info.get('Good_with_children', 'N/A')
                cats = dog_info.get('Good_with_cats', 'N/A')
                other_dogs = dog_info.get('Good_with_dogs', 'N/A')
                other_animals = dog_info.get('Good_with_other_animals', 'N/A')
                fee_waived = dog_info.get('Adoption_fee_waived', 'N/A')

                # Extracting desired field names not in dog_info 
                try:
                    pet_location = driver.find_element('xpath', '//span[@data-test="Pet_Location"]').text
                except:
                    pet_location = 'N/A'

                try:
                    characteristics = driver.find_element('xpath', '//dt[contains(text(), "Characteristics")]/following-sibling::dd').text
                except:
                    characteristics = 'N/A'

                try:
                    house_trained = driver.find_element('xpath', '//dt[contains(text(), "House-trained")]/following-sibling::dd').text
                except:
                    house_trained = 'N/A'

                try:
                    health = driver.find_element('xpath', '//dt[contains(text(), "Health")]/following-sibling::dd').text
                except:
                    health = 'N/A'

                try:
                    adoption_fee = driver.find_element('xpath', '//dt[contains(text(), "Adoption fee")]/following-sibling::dd').text
                except:
                    try:
                        pet_story_section = driver.find_element('xpath', '//div[@data-test="Pet_Story_Section"]').text
                        adoption_fee_search = re.search(r'Adoption fee (\d+)', pet_story_section, re.IGNORECASE)

                        if adoption_fee_search:
                            adoption_fee = adoption_fee_search.group(1)
                        else:
                            adoption_fee = 'N/A'  # If the regex doesn't find a match
                    except:
                        # If neither method works, set the adoption fee to 'N/A'
                        adoption_fee = 'N/A'

                dog_data_list.append({
                    'pet_id': pet_id,
                    'pet_name': pet_name,
                    'primary_breed': primary_breed,
                    'secondary_breed': secondary_breed,
                    'mixed_breed': mixed_breed,
                    'age': age,
                    'gender': gender,
                    'size': size,
                    'primary_colour': primary_colour,
                    'secondary_colour': secondary_colour,
                    'coat_length': coat_length,
                    'shelter_name': shelter_name,
                    'shelter_id': shelter_id,
                    'zip_code': zip_code,
                    'num_photos': num_photos,
                    'children': children,
                    'cats': cats,
                    'other_dogs': other_dogs,
                    'other_animals': other_animals,
                    'fee_waived': fee_waived,
                    'pet_location': pet_location,
                    'characteristics': characteristics,
                    'house_trained': house_trained,
                    'health': health,
                    'adoption_fee': adoption_fee
                })

                # Navigate back to the results page
                driver.back()

                # Wait for the results page to reload
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="petCard-link"]'))
                )
                
                # Increment the counter for processed dogs
                processed_dogs += 1

            except Exception as e:
                print(f"Error processing dog {processed_dogs}: {e}")
                break
        return dog_data_list
    finally:
        # Close the browser once finished
        driver.quit()